# 키워드 기반 구글 이미지 수집

In [16]:
# pip install webdriver_manager

### 1. 라이브러리 선언

In [17]:
# 브라우저 자동화 라이브러리
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 날짜계산 라이브러리 (파일명 생성용)
from datetime import datetime
import os, time

# 웹 이미지 다운로드 위한 라이브러리
import urllib.request

In [18]:
def createFolder(directory): 
    try: 
        if not os.path.exists(directory): 
            os.makedirs(directory) 
    except OSError: 
        print ('Error: Creating directory. ' + directory)

### 2. 키워드 및 저장폴더 설정

In [19]:
search_word = "고궁 낮"

today_time = datetime.now().strftime("%Y%m%d_%H%M")
file_path = "./images/"

# 최종저장 폴더 생성
finalFolder = file_path + today_time + '_' + search_word
createFolder(finalFolder)
print(finalFolder)

./images/20221118_1449_고궁 낮


### 3. 셀레니움 활용 브라우저 기동

In [20]:
def setChromeDriver():
    options = Options()
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'
    options.add_argument('user-agent=' + user_agent)
    # options.add_argument('--headless')  # 웹 브라우저를 시각적으로 띄우지 않는 headless chrome 옵션
    driver = webdriver.Chrome(service=Service(executable_path=ChromeDriverManager().install()), options=options)
    return driver

driver = setChromeDriver()

### 4. 키워드 이미지 탐색

In [21]:
### 브라우저 오픈 및 구글 페이지로 이동
url = 'https://www.google.co.kr'
driver.get(url)
time.sleep(3)

### 검색키워드 입력
searchBar = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
searchBar.send_keys(search_word)
searchBar.send_keys(Keys.ENTER)

### 이미지 찾기 전환
imageTabXpath = '//*[@id="hdtb-msb"]/div[1]/div/div[2]/a'
driver.find_element(By.XPATH, imageTabXpath).click()  # 텍스트 메뉴 '이미지' 링크 클릭
time.sleep(3)

### 5. 페이지 스크롤링

In [22]:
elem = driver.find_element(By.TAG_NAME, 'body')
page_height = driver.execute_script('return document.body.scrollHeight')
pg_down_cnt = 1

while True:
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(3)
    print("page_down_cnt= {}".format(pg_down_cnt))
    try:
        pg_down_cnt = pg_down_cnt + 1 
        driver.find_element(By.XPATH, '//*[@id="islmp"]/div/div/div/div[1]/div[2]/div[2]/input').click()  # 결과 더보기 버튼 처리
        print('결과 더보기 버튼 클릭 처리')
    except:
        if pg_down_cnt > 3: 
            break
        continue

page_down_cnt= 1
page_down_cnt= 2
page_down_cnt= 3


### 6. 작은 이미지 링크 수집

In [23]:
imgs = driver.find_elements(By.CLASS_NAME, "rg_i")

### 7. 실제 이미지 링크 수집

In [24]:
count = 1
img_src = []

for i in range(0, len(imgs)):
    img = imgs[i]
    img.click()  # 이미지 클릭 시 display 되는 url을 찾기 위해 클릭함
    try:
        bigImgXpath = '//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[3]/div/a/img'
        actualImgTag = driver.find_element(By.XPATH, bigImgXpath)
    except Exception:
        continue
        
    time.sleep(1)
    actualImgSrc = actualImgTag.get_attribute('src')

    if actualImgSrc[:4] != 'http':
        continue

    print(count, actualImgSrc, '\n')

    img_src.append(actualImgSrc)     
    count = count + 1

### 8. 이미지 파일 다운로드

In [25]:
finalImgLen = len(img_src)

In [26]:
cnt=0

for i in range(0, finalImgLen):

    bigImage_url = img_src[i]
    file_ext = bigImage_url.split('.')[-1]
    if( file_ext in ['jpg', 'jpeg', 'png', 'gif', 'bmp']):
        file_ext = "." + file_ext
        pass
    else:
        file_ext = ".jpg"
    
    try:
        filename = search_word + str(i).zfill(3) + file_ext
        # 파일을 디렉토리에 저장
        urllib.request.urlretrieve(bigImage_url, finalFolder + '/' + filename)
        cnt = cnt + 1 
        print( "{} 완료".format(cnt) )
    except Exception as e:
        print(i, e)